In [1]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

In [3]:
def open_jsonl(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return [json.loads(line.strip()) for line in lines]

In [4]:
dataset_path = "/home/doyooni303/experiments/sequential_recsys/datasets/amazon"
dataset_name = "Books"
history = json.load(open(os.path.join(dataset_path,dataset_name,f"{dataset_name}_history.json"),'r'))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8c6325f520>>
Traceback (most recent call last):
  File "/root/.local/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
itemidx2meta = json.load(open(os.path.join(dataset_path,dataset_name,f"{dataset_name}_itemidx2meta.json"),'r'))

NameError: name 'json' is not defined

### Longformer로 우선 아이템 임베딩 구해보기

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np

### Item text부터 만들기

In [ ]:
def get_text(meta: dict, keys: list):
    text = ""
    for key in keys:
        try:
            if isinstance(meta[key],str):
                text += f"{key}: {meta[key]}\n"
            elif isinstance(meta[key],list):
                if len(meta[key])>0:
                    add = ", ".join(meta[key])
                    text += f"{key}: {add}\n"
            
            if key=='author':
                text += f"author: {meta['author']['name']}\n"
        except:
            text += f"{key}: None\n"
    return text



In [ ]:
meta = itemidx2meta['1']
meta_keys= ['main_category',
 'title',
 'subtitle',
 'author',
 'average_rating',
 'features',
 'description',
 'categories',]

text = get_text(meta, meta_keys)
print(text)

main_category: Books
title: Notes from a Kidwatcher
subtitle: First Edition
author: Yetta M. Goodman
features: Contains 23 selected articles by this influential writer, researcher, educator, and speaker. They're grouped around six major themes inherent in teacher education: culture and community; miscue analysis, reading strategies and comprehension; print awareness and the roots of literacy; the writing process; kidwatching; and whole language theory. No index. Annotation c. by Book News, Inc., Portland, Or.
description: About the Author, SANDRA WILDE, Ph.D., is widely recognized for her expertise in developmental spelling and her advocacy of holistic approaches to spelling and phonics. She is Professor of Curriculum and Instruction at Portland State University in Oregon. She is best known for her work in invented spelling, phonics and miscue analysis. She specializes in showing teachers how kids' invented spellings and miscues can help us work with them in more sophisticated and lear

In [ ]:
def save_jsonl(data: list, path: str, desc) -> None:
    with open(path, "w", encoding="utf-8") as f:
        for line in tqdm(data, desc=desc):
            json.dump(line, f, ensure_ascii=False)
            f.writelines("\n")

item_texts = []
for idx, meta in tqdm(itemidx2meta.items(), total=len(itemidx2meta)):
    item_dict = {}
    text = get_text(meta, meta_keys)
    item_dict['idx'] = idx
    item_dict['text'] = text
    item_texts.append(item_dict)

# save_jsonl(item_texts,
#            os.path.join(dataset_path,dataset_name,f"{dataset_name}_item_texts.jsonl"),
#            desc="Saving item texts")

100%|██████████| 4448181/4448181 [00:42<00:00, 105369.33it/s]


In [ ]:
import torch
import faiss
import numpy as np
from typing import Tuple, List

class SimilaritySearch:
    def __init__(self, embedding_dim: int = 768):
        self.embedding_dim = embedding_dim
        self.index = None
        self.item_id_map = None
    
    def build_index(self, embedding_system, custom_ids=None, device='cpu'):
        """Build FAISS index from embedding layer"""
        # Get all embeddings from the embedding layer
        with torch.no_grad():  # We don't need gradients for index building
            embeddings = embedding_system.item_embeddings.weight.data
            if device == 'cpu':
                embeddings = embeddings.cpu().numpy()
            else:
                embeddings = embeddings.cuda().numpy()
        
        # Normalize embeddings (optional, but recommended for cosine similarity)
        faiss.normalize_L2(embeddings)
        
        # Create FAISS index
        self.index = faiss.IndexFlatIP(self.embedding_dim)  # Inner product for cosine similarity
        self.index.add(embeddings)
        
        # Set up custom ID mapping if provided
        if custom_ids is not None:
            if len(custom_ids) != len(embeddings):
                raise ValueError(f"Length of custom_ids ({len(custom_ids)}) must match number of embeddings ({len(embeddings)})")
            self.item_id_map = {i: custom_id for i, custom_id in enumerate(custom_ids)}
        else:
            self.item_id_map = {i: i for i in range(len(embeddings))}
        
        # Create mapping from index position to item_id
        # This can be customized based on your item IDs
        self.item_id_map = None  # Will be set when custom IDs are provided
        
        return self
    
    def update_specific_embeddings(self, embeddings: torch.Tensor, indices: List[int]):
        """
        Update specific embeddings in the FAISS index
        Args:
            embeddings: tensor of shape (n, embedding_dim) containing updated embeddings
            indices: list of indices corresponding to the embeddings to update
        """
        if self.index is None:
            raise ValueError("Index not built yet! Call build_index first.")
            
        # Convert embeddings to numpy
        updated_embeddings = embeddings.detach().cpu().numpy()
        
        # Normalize the updated embeddings
        faiss.normalize_L2(updated_embeddings)
        
        # Remove old vectors
        self.index.remove_ids(np.array(indices))
        
        # Add updated vectors
        self.index.add_with_ids(updated_embeddings, np.array(indices))
        
    def search(self, query_embedding: torch.Tensor, k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """
        Search for k most similar items
        Args:
            query_embedding: tensor of shape (1, embedding_dim)
            k: number of similar items to retrieve
        Returns:
            distances, item_indices
        """
        if self.index is None:
            raise ValueError("Index not built yet! Call build_index first.")
            
        # Convert query to numpy and ensure correct shape
        query = query_embedding.detach().cpu().numpy()
        if len(query.shape) == 1:
            query = query.reshape(1, -1)
            
        # Normalize query (if you normalized the index)
        faiss.normalize_L2(query)
        
        # Search
        distances, indices = self.index.search(query, k)
        
        # Map indices to original item ids
        item_indices = np.array([self.item_id_map[idx] for idx in indices[0]])
        
        return distances[0], item_indices

def example_usage():
    # Assuming you have your embedding system ready
    embedding_system = ItemEmbeddingSystem(num_items=747800)
    
    # Build similarity search index
    similarity_search = SimilaritySearch(embedding_dim=768)
    similarity_search.build_index(embedding_system)
    
    # Example: Get embedding for a query item
    query_idx = torch.tensor([42])  # example item id
    query_embedding = embedding_system(query_idx)
    
    # Find similar items
    distances, similar_item_indices = similarity_search.search(query_embedding[0], k=10)
    
    # Print results
    print("Query item:", query_idx.item())
    print("Similar items (id, distance):")
    for idx, dist in zip(similar_item_indices, distances):
        print(f"Item {idx}: {dist:.4f}")
    
    return similar_item_indices, distances

# Usage during training
class RecommenderWithFAISS:
    def __init__(self, embedding_system, rebuild_index_frequency=100):
        self.embedding_system = embedding_system
        self.similarity_search = SimilaritySearch(embedding_dim=768)
        self.rebuild_index_frequency = rebuild_index_frequency
        self.training_step = 0
        
    def update_and_search(self, query_idx: int, k: int = 5):
        """Find similar items and update their embeddings after training"""
        # 1. Get initial similar items
        similar_items, distances = self.find_similar_items(query_idx, k)
        
        # 2. Get embeddings for these items
        item_indices = torch.tensor(similar_items)
        original_embeddings = self.embedding_system(item_indices)
        
        # 3. Perform training step (example with dummy labels)
        self.embedding_system.train()
        loss = self.train_step(item_indices, torch.zeros_like(item_indices))
        
        # 4. Get updated embeddings
        self.embedding_system.eval()
        with torch.no_grad():
            updated_embeddings = self.embedding_system(item_indices)
        
        # 5. Update these specific embeddings in FAISS
        self.similarity_search.update_specific_embeddings(
            updated_embeddings,
            similar_items.tolist()
        )
        
        return similar_items, original_embeddings, updated_embeddings, loss

    def train_step(self, item_indices, labels):
        # Regular training step
        self.embedding_system.train()
        embeddings = self.embedding_system(item_indices)
        loss = self.calculate_loss(embeddings, labels)
        loss.backward()
        
        # Rebuild FAISS index periodically
        self.training_step += 1
        if self.training_step % self.rebuild_index_frequency == 0:
            self.embedding_system.eval()
            self.similarity_search.build_index(self.embedding_system)
        
        return loss
    
    def find_similar_items(self, query_idx: int, k: int = 10):
        """Find similar items using FAISS"""
        self.embedding_system.eval()
        with torch.no_grad():
            query_embedding = self.embedding_system(torch.tensor([query_idx]))
        
        return self.similarity_search.search(query_embedding[0], k=k)

In [22]:
### longformer 이용하는 경우

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is on {device}")
longformer = AutoModel.from_pretrained("allenai/longformer-base-4096").to(device)
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

batch_size = 12
item_embeddings= []
item_idxs = []
for i in tqdm(range(0, len(item_texts), batch_size)):
    batches = item_texts[i:i+batch_size]
    idxs = [meta['idx'] for meta in batches]
    texts = [meta['text'] for meta in batches]

    inputs = tokenizer(texts, return_tensors="pt", padding="max_length", max_length=4096, truncation=True).to(device)
    longformer.eval()
    outputs = longformer(**inputs)
    embeddings = outputs.last_hidden_state[:,0,:].detach().cpu().tolist()
    item_idxs.extend(idxs)
    item_embeddings.extend(embeddings)
    del inputs, outputs
    break

Device is on cuda:3


/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  0%|          | 0/370682 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.11 GiB. GPU 3 has a total capacity of 79.15 GiB of which 1.47 GiB is free. Process 2088314 has 77.65 GiB memory in use. Of the allocated memory 73.79 GiB is allocated by PyTorch, and 3.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [12]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Device is on {device}")
longformer = AutoModel.from_pretrained("allenai/longformer-base-4096").to(device)
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

{'input_ids': tensor([[    0, 17894,  1215,  ...,     1,     1,     1],
        [    0, 17894,  1215,  ...,     1,     1,     1],
        [    0, 17894,  1215,  ...,     1,     1,     1],
        ...,
        [    0, 17894,  1215,  ...,     1,     1,     1],
        [    0, 17894,  1215,  ...,     1,     1,     1],
        [    0, 17894,  1215,  ...,     1,     1,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import JSONLoader

max_length = 384
model_kwargs = {'device': device}
encode_kwargs = {'padding': 'max_length', 'max_length':max_length, 'return_tensors': 'pt', 'truncation': True}
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2',
                                        model_kwargs=model_kwargs,
                                        encode_kwargs=encode_kwargs,
                                        )
loader = JSONLoader(file_path=os.path.join(dataset_path,dataset_name,f"{dataset_name}_item_texts.jsonl"),
                    jq_schema=".text",
                    text_content=False,
                    json_lines=True)
data = loader.load()

# index=faiss.IndexFlatL2(embeddings.)
# index
# db = FAISS(
#     embedding_function=embeddings,
#     index=
# )

In [ ]:
input_ids = tokenizer(text,
          padding='max_length',
          max_length=4096,
          truncation=True,
          return_tensors='pt').to(device)
outputs = longformer(**input_ids)
outputs.pooler_output.squeeze().detach().cpu().tolist()

[0.1259765625,
 -0.26171875,
 0.0947265625,
 0.04638671875,
 0.345703125,
 -0.205078125,
 -0.50390625,
 -0.443359375,
 -0.115234375,
 -0.25,
 -0.353515625,
 -0.185546875,
 0.1953125,
 -0.34375,
 0.095703125,
 -0.1865234375,
 -0.365234375,
 -0.0888671875,
 -0.1005859375,
 -0.0400390625,
 -0.0252685546875,
 -0.015869140625,
 -0.2451171875,
 -0.0015716552734375,
 -0.4140625,
 -0.0068359375,
 -0.1728515625,
 -0.244140625,
 0.234375,
 0.2451171875,
 -0.06982421875,
 -0.003204345703125,
 -0.3359375,
 0.006500244140625,
 -0.0869140625,
 0.2490234375,
 -0.17578125,
 0.1611328125,
 -0.396484375,
 0.2470703125,
 0.02001953125,
 0.2294921875,
 0.263671875,
 0.0186767578125,
 0.1669921875,
 0.17578125,
 -0.2119140625,
 -0.310546875,
 -0.263671875,
 0.474609375,
 0.291015625,
 0.404296875,
 0.29296875,
 0.1005859375,
 -0.251953125,
 -0.1591796875,
 -0.06884765625,
 0.0703125,
 -0.1552734375,
 -0.1787109375,
 -0.28125,
 -0.2490234375,
 -0.062255859375,
 -0.2021484375,
 -0.0712890625,
 0.150390625,
 

In [ ]:
from langchain_core.prompts import PromptTemplate
template = """Main Category: {main_category}
Title: {title}
Subtitle: {subtitle}
Author: {author}
Average Rating: {average_rating}
Features: {features}
Description: {description}
Categories: {categories}
"""

prompt = PromptTemplate.from_template(template)
prompt.invoke(input=meta)


StringPromptValue(text="Main Category: Books\nTitle: Chaucer\nSubtitle: Hardcover – Import, January 1, 2004\nAuthor: {'avatar': 'https://m.media-amazon.com/images/I/21Je2zja9pL._SY600_.jpg', 'name': 'Peter Ackroyd', 'about': ['Peter Ackroyd, (born 5 October 1949) is an English biographer, novelist and critic with a particular interest in the history and culture of London. For his novels about English history and culture and his biographies of, among others, William Blake, Charles Dickens, T. S. Eliot and Sir Thomas More, he won the Somerset Maugham Award and two Whitbread Awards. He is noted for the volume of work he has produced, the range of styles therein, his skill at assuming different voices and the depth of his research.', 'He was elected a fellow of the Royal Society of Literature in 1984 and appointed a Commander of the Order of the British Empire in 2003.', 'Bio from Wikipedia, the free encyclopedia.']}\nAverage Rating: 4.5\nFeatures: []\nDescription: []\nCategories: ['Books'